In [28]:
import ctf,time,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()

In [18]:
class UnitTests:
    def test3d(self):
        print("pass")
        pass
        
    def runAllTests(self):
        self.test3d()

In [26]:
def normalize(Z,r):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [7]:
def updateOmega(I,J,K,sparsity):
    '''
    Gets a random subset of rows for each U,V,W iteration
    '''
    Actf = ctf.tensor((I,J,K),sp=True)
    Actf.fill_sp_random(0,1,sparsity)
    omegactf = ((Actf > 0)*ctf.astensor(1.))
    return omegactf


def getIndexFromOmega(omega,I,J,K):
    '''
    Gets a random i,j,k contained in Ω[0...i-1,0...j-1,0...k-1]
    '''
    #TODO: Better sampling instead of MC.
    i = random.randint(0,I-1)
    j = random.randint(0,J-1)
    k = random.randint(0,K-1)
    while omega[i][j][k] != 1:
        i = random.randint(0,I-1)
        j = random.randint(0,J-1)
        k = random.randint(0,K-1)
        
    return (i,j,k)

In [34]:
def updateU(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update U matrix by using the formula'''
    
    M1 = ctf.tensor((J,K,r))
    M1.i("jku") << V.i("ju")*W.i("ku")
    [U_,S_,V_] = ctf.svd(M1.reshape((J*K,r)))
    S_ = 1./S_
    U.set_zero()
    U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
    normalize(U,r)
    
    return U   
    
    
def updateV(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M2 = ctf.tensor((I,K,r))
    M2.i("iku") << U.i("iu")*W.i("ku")
    [U_,S_,V_] = ctf.svd(M2.reshape((I*K,r)))
    S_ = 1./S_
    V.set_zero()
    V.i("ju") << V_.i("vu")*S_.i("v")*U_.reshape((I,K,r)).i("ikv")*T.i("ijk")
    normalize(V,r)
    
    return V   

def updateW(T,U,V,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M3 = ctf.tensor((I,J,r))
    M3.i("iju") << U.i("iu")*V.i("ju")
    [U_,S_,V_] = ctf.svd(M3.reshape((I*J,r)))
    S_ = 1./S_
    W.set_zero()
    W.i("ku") << V_.i("vu")*S_.i("v")*U_.reshape((I,J,r)).i("ijv")*T.i("ijk")
    normalize(W,r)
    
    return W

In [35]:
def getALSCtf(T,U,V,W,regParam,omega,I,J,K,r):
    '''
    Same thing as above, but CTF
    '''
    it = 0
    E = ctf.tensor((I,J,K))
    E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
    curr_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
    
    while True:
        U = updateU(T,U,V,W,regParam,omega,I,J,K,r) 
        V = updateV(T,U,V,W,regParam,omega,I,J,K,r) 
        W = updateW(T,U,V,W,regParam,omega,I,J,K,r) 
        E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
        next_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
        
        if curr_err_norm - next_err_norm > .001 or it > 10:
            break
            
        print(curr_err_norm, next_err_norm)
        curr_err_norm = next_err_norm
        it += 1
        
    return U,V,W

In [36]:
def main():
    
    ut = UnitTests()
    ut.runAllTests()

    I = random.randint(1,5)
    J = random.randint(1,5)
    K = random.randint(1,5)
    r = 3 
    sparsity = .2
    regParam = 2
    
    # 3rd-order tensor
    T = ctf.tensor((I,J,K),sp=True)
    T.fill_sp_random(0,1,1)
    
    ctf.random.seed(42)
    U = ctf.random.random((I,r))
    V= ctf.random.random((J,r))
    W= ctf.random.random((K,r))
    
    normalize(U,r)
    normalize(V,r)
    normalize(W,r)
    
    omega = updateOmega(I,J,K,sparsity)
    
    t = time.time()
    
    getALSCtf(T,U,V,W,regParam,omega,I,J,K,r)
    
    print("ALS costs time = ",np.round_(time.time()- t,4))    

In [37]:
main()

pass
13.783784782178587 17.06726909747425
17.06726909747425 20.331349023876886
20.331349023876886 23.553564019727894
23.553564019727894 26.74816900604123
26.74816900604123 29.937345974958337
29.937345974958337 33.12394729221492
33.12394729221492 36.30575747410268
36.30575747410268 39.480612168969365
39.480612168969365 42.64716978125728
42.64716978125728 45.80527584707603
45.80527584707603 48.956220455790884
ALS costs time =  2.0665
